### After estimating the cell type abundance individually on each patient (grouped per replicate), we can look at the distrribution of cell types across the pathological annootations.  And look for enrichment of those cell types across certain regions?

In [1]:
from pathlib import Path
import scanpy as sc
import cell2location
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as stats

DPI = 300
FONTSIZE = 20  # 42
sc.settings.set_figure_params(
    scanpy=True, dpi=100, transparent=True, vector_friendly=True, dpi_save=DPI
)
from matplotlib import rcParams

rcParams["pdf.fonttype"] = 42

from vistools import utils

Global seed set to 0


In [2]:
SAMPLE_LIST = ['SN048_A416371', 'SN84_A120838', 'SN123_A595688']

In [3]:
SAMPLE_NAME = '_'.join([s for s in SAMPLE_LIST])
SAMPLE_NAME

'SN048_A416371_SN84_A120838_SN123_A595688'

In [4]:
DIR2SAVE = Path(f"/data/BCI-CRC/nasrine/data/CRC/spatial/public/Visium_Valdeolivas_2023/cell2loc_pathologist_annotations/{SAMPLE_NAME}/")
DIR2SAVE.mkdir(exist_ok=True, parents=True)
FIG2SAVE = DIR2SAVE.joinpath("figures/")
FIG2SAVE.mkdir(exist_ok=True, parents=True)

In [5]:
slides = dict()
for s in SAMPLE_LIST:
# load in gene matrix so we get sample name?
    slides[s] = sc.read_h5ad(
    f"/data/BCI-CRC/nasrine/data/CRC/spatial/public/Visium_Valdeolivas_2023/cell2location/{s}/cell2location_map-no_cycling_TME/sp.h5ad"
)

In [6]:
slides.keys()

dict_keys(['SN048_A416371', 'SN84_A120838', 'SN123_A595688'])

In [7]:
# create variable celltypes list for which we have mRNA abundance and will compute average abundance/region later : 
celltype_list = list()

for slide in slides:
    # add mRNA counts to data
    slides[slide].obsm["q05_mRNA_abundance_u_sf"] = pd.DataFrame(
        slides[slide].uns["mod"]["post_sample_q05"]["u_sf_mRNA_factors"].round().astype(np.int32),
        index=slides[slide].obs_names,
        columns=[
        f"q05_mRNA_abundance_u_sf_{i}" for i in slides[slide].uns["mod"]["factor_names"]
    ])
    
    mrna_abundance_df = pd.DataFrame(data=slides[slide].uns["mod"]["post_sample_q05"]["u_sf_mRNA_factors"].round().astype(np.int32),
                                 index=slides[slide].obs_names,
                                 columns=[f"{i}" for i in slides[slide].uns["mod"]["factor_names"]])
    
    # merge cell mRNA abundance to obs
    slides[slide].obs = slides[slide].obs.merge(mrna_abundance_df, how='left', left_index=True, right_index=True)
    
    # merge all pathologist annotations into one single column
    if 'Pathologist annotation' in slides[slide].obs.columns:
        slides[slide].obs['Pathologist Annotation grouped'] = slides[slide].obs.loc[:, ['Pathologist annotation',
                                                                                        'Pathologist Annotation']
                                                                               ].apply(
            lambda x: ''.join(x.dropna().astype(str)), axis=1
        )
        
    # merge all pathologist annotations into one single column
    if 'Pathologist Annotations' in slides[slide].obs.columns:
        slides[slide].obs['Pathologist Annotation grouped'] = slides[slide].obs.loc[:, ['Pathologist Annotation',
                                                                                        'Pathologist Annotations']
                                                                               ].apply(
            lambda x: ''.join(x.dropna().astype(str)), axis=1
        )
    
    # merge all pathologist annotations into one single column
    if slide=='SN123_A595688':
        slides[slide].obs['Pathologist Annotation grouped'] = slides[slide].obs.loc[:, 'Pathologist Annotation']
    
    # cell types list:
    celltype_list = [f"{i}" for i in slides[slide].uns["mod"]["factor_names"]]


In [8]:
df = pd.concat([slides[slide].obs for slide in slides], axis=0)

In [9]:
df.head(3)

,in_tissue,array_row,array_col,Sample,n_genes_by_counts,total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,...,UPR,cDC1,cDC2,gdT,ipEMT,migDC,pDC,pEMT,Pathologist Annotation grouped,Pathologist Annotations
spot_id,,,,,,,,,,,,,,,,,,,,,
Count_SN048_A416371_Rep1_AAACAAGTATCTCCCA-1,1,50,102,Count_SN048_A416371_Rep1,3447,6769.0,15.068696,22.972374,32.323829,47.215246,...,53,24,4,16,1958,22,2,915,tumor&stroma_IC med to high,NaN
Count_SN048_A416371_Rep1_AAACACCAATAACTGC-1,1,59,19,Count_SN048_A416371_Rep1,3706,8120.0,16.293103,24.445813,33.953202,48.374384,...,32,211,11,10,8041,15,5,78,tumor&stroma_IC med to high,NaN
Count_SN048_A416371_Rep1_AAACAGAGCGACTCCT-1,1,14,94,Count_SN048_A416371_Rep1,3809,7946.0,14.510446,22.048830,30.820539,45.117040,...,4,2,1,0,7,2,1,1,tumor&stroma_IC med to high,NaN


In [10]:
df.groupby('Sample').size()

Sample
Count_SN048_A416371_Rep1    2317
Count_SN048_A416371_Rep2    1803
Count_SN123_A595688_Rep1     649
Count_SN124_A595688_Rep2     283
Count_SN84_A120838_Rep1      328
Count_SN84_A120838_Rep2     1048
dtype: int64

In [10]:
df.to_csv(DIR2SAVE.joinpath("cell2loc_mrna_abundance_pathological_annotations.csv"),
          sep='\t',
          header=True,
          index=True
         )

In [11]:
DIR2SAVE

PosixPath('/data/BCI-CRC/nasrine/data/CRC/spatial/public/Visium_Valdeolivas_2023/cell2loc_pathologist_annotations/SN048_A416371_SN84_A120838_SN123_A595688')

In [15]:
pd.Series(celltype_list).to_csv(DIR2SAVE.joinpath("celltypes.csv"),
          sep='\t',
          header=False,
          index=False)